# Computer vision in python

El objetivo de este notebook es mostrar por un lado la capacidad de ciertas librerías de python en torno al _data augmentation_ y por otro lado aplicar esta técnica en un caso real de reconocimiento de objetos.

Para ello, vamos a generar de forma aleatoria a partir de 5 imágenes básicas un conjunto de 250 imágenes, etiquetarlas y entrenar un modelo para luego realizar detección sobre dicho modelo.

## Primer paso: Data augmentation

Esta técnica se usa comúnmente en técnicas de visión computacional cuando los datos de los que disponemos no son suficientes para nuestro estudio. A _grosso modo_, la técnica consiste en alterar una imagen de forma que para el entrenamiento de nuestro modelo, pueda contar como una imagen diferente y que por tanto aporta información extra al modelo.

In [6]:
# Cargamos las librerías

import os
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import random
from PIL import Image
from numpy import *
from sympy import *

In [9]:
# Cargamos las 5 imágenes que vamos a emplear

originales = {}

for img_name in os.listdir("./originales"):
    name = img_name.split('.')[0]
    originales[name] = imageio.imread("./originales/"+img_name)

In [3]:
def random_image(img):
    """
    Esta función toma como argumento una imagen a la que de forma aleatoria se le van a aplicar con probabilidad 0.25 una de las siguientes transformaciones:
    - Rotación.
    - Rotación y reflexión.
    - Rotación y ruido.
    - Torsión.
    
    Todas ellas también con un grado aleatorio de alteración.
    """
    x = random.randint(-50, 50)
    y = random.randint(-50, 50)
    z =  random.randint(1, 80)
    if z <= 25: # rotamos la imagen
        rotate=iaa.Affine(rotate=(x, y))
        return rotate.augment_image(img)
    elif z <= 50: # la rotamos y le hacemos una reflexión sobre el eje vertical
        flip_hr=iaa.Fliplr(p=1.0)
        rotate=iaa.Affine(rotate=(x, y))
        return flip_hr.augment_image(rotate.augment_image(img))
    elif z <= 75: # la rotamos y le agregamos ruido a la imagen
        x = random.randint(-80, 80)
        y = random.randint(-80, 80)
        gaussian_noise=iaa.AdditiveGaussianNoise(-60,60)
        rotate=iaa.Affine(rotate=(x, y))
        return gaussian_noise.augment_image(rotate.augment_image(img))
    else: # torcemos la imagen
        x = random.randint(30, 60)
        y = random.randint(30, 60)
        shear = iaa.Affine(shear=(x,y))
        return shear.augment_image(img)

In [14]:
# Generamos las 250 imágenes para el etiquetado y entrenamiento
for i in originales.keys():
    for j in range(50):
        filename = "random_image_{}_{}.jpg".format(j,i)
        imageio.imwrite("train/images/"+filename, random_image(originales[i])) 

## Segundo paso: Etiquetado

Necesitamos etiquetar de forma que el modelo pueda _aprender_ de las imágenes. Para ello, vamos a emplear una herramienta